In [ ]:
%load_ext autoreload
%autoreload 2
import waffles
import numpy as np
import json
import shutil 
from tqdm import tqdm

from waffles.input_output.hdf5_structured import load_structured_waveformset
from waffles.data_classes.Waveform import Waveform
from waffles.data_classes.WaveformSet import WaveformSet
from waffles.data_classes.BasicWfAna import BasicWfAna
from waffles.data_classes.IPDict import IPDict
from waffles.data_classes.UniqueChannel import UniqueChannel
from waffles.data_classes.ChannelWsGrid import ChannelWsGrid
from waffles.utils.baseline.baseline import SBaseline
from waffles.np02_utils.AutoMap import generate_ChannelMap, dict_uniqch_to_module, dict_module_to_uniqch
from waffles.np02_utils.PlotUtils import np02_gen_grids, plot_grid, plot_detectors, genhist, fithist, runBasicWfAnaNP02
from waffles.np02_utils.load_utils import open_processed

In [ ]:
run = 41470
nwaveforms = 80000
dettype = "pmt"

## Only change if necessary
output_dir = f"/eos/experiment/neutplatform/protodune/experiments/ProtoDUNE-VD/commissioning/"
ch = {}
det = "VD_PMT_PDS" 
endpoint = 110
trigger = "self_trigger"

In [ ]:
wfset_full = open_processed(run, dettype, output_dir, nwaveforms=nwaveforms)
wfset_full

In [ ]:
from collections import Counter
timestamps = sorted([ wf.timestamp for wf in wfset_full.waveforms ])
c = Counter(timestamps)
print(f"Total number of timestamps: {len(c)}")
matchtimestamps = [ k for k in c if c[k] >= len(list(wfset_full.available_channels[run][endpoint]))
                    1]
# matchtimestamps = [ k for k in c if c[k] >= 20 ]
print(f"Remaining timestamps: {len(matchtimestamps)}")

In [ ]:
def get_external(waveform: Waveform, validtimes = []) -> bool:
    if np.any((waveform.adcs[0:500] > 16000) | (waveform.adcs[0:500] < 100) ):
        return False

    if waveform.timestamp not in validtimes:
        return False
    
    return True

matchingtimes = []
print(wfset_full)
wfset_full = WaveformSet.from_filtered_WaveformSet(wfset_full, get_external, matchtimestamps, show_progress=True)
print(wfset_full)

In [ ]:
runBasicWfAnaNP02(wfset_full, baselinefinish=240, onlyoptimal=True, threshold=25)

In [ ]:
argsheat = dict(
    mode="heatmap",
    analysis_label="std",
    adc_range_above_baseline=1000,
    adc_range_below_baseline=-200,
    adc_bins=200,
    time_bins=wfset_full.points_per_wf//2,
    filtering=4,
    share_y_scale=False,
    share_x_scale=True,
    wfs_per_axes=50,
    zlog=True,
    width=1600,
)
pmtuch = list(wfset_full.available_channels[run][110])
detector=[ UniqueChannel(110, ch) for ch in pmtuch ]
# plot_detectors(wfset_full, detector, plot_function=fithist, html=None, wf_func=histargs)
plot_detectors(wfset_full, detector, **argsheat)
          
# detector=[ UniqueChannel(110, ch) for ch in pmtuch[:len(pmtuch)//2] ]
# plot_detectors(wfset_full, detector, **argsheat)
# detector=[ UniqueChannel(110, ch) for ch in pmtuch[len(pmtuch)//2:] ]
# plot_detectors(wfset_full, detector, **argsheat)

In [ ]:
histargs = dict(    
    # doprocess = False,
    dofit = True,
    variable = 'integral',
    show_progress = False,
    configyaml="ch_snr_parameters_pmts.yaml",
    fitmultigauss=False,
    log_y=True,
    # normalize_hist=True,
)
argsplot = dict(
    width=1600
)
pmtuch = list(wfset_full.available_channels[run][endpoint])
# detector=[ UniqueChannel(110, ch) for ch in pmtuch ]
# detector=[ UniqueChannel(endpoint, 12) ]

detector=[ UniqueChannel(endpoint, ch) for ch in pmtuch[:len(pmtuch)//2] ]
plot_detectors(wfset_full, detector, plot_function=fithist, wf_func=histargs, **argsplot)

detector=[ UniqueChannel(endpoint, ch) for ch in pmtuch[len(pmtuch)//2:] ]
plot_detectors(wfset_full, detector, plot_function=fithist, wf_func=histargs, **argsplot)

In [ ]:
def remove_bad_baseline(waveform:Waveform) -> bool:
    # comment the two lines below to not remove any waveform
    if 'std' in waveform.analyses and waveform.analyses["std"].result['amplitude'] is np.nan:
        return True
    return True
    
wfset = WaveformSet.from_filtered_WaveformSet(wfset_full, remove_bad_baseline)


print(f"After removing: {len(wfset.waveforms)} waveforms")
wfch_full = ChannelWsGrid.clusterize_waveform_set(wfset_full)
wfch = ChannelWsGrid.clusterize_waveform_set(wfset)
print("By channel..")
for ep, v in wfch.items():
    print("Endpoint:", ep)
    for ch, wfs in v.items():
        print(f"Ch: {ch}, {len(wfs.waveforms)}, total: {len(wfch_full[ep][ch].waveforms)}")

In [ ]:
from waffles.np02_utils.PlotUtils import matplotlib_plot_WaveformSetGrid

In [ ]:
detectors = []
import matplotlib.pyplot as plt
for ch in wfset_full.available_channels[run][endpoint]:
    detectors.append(UniqueChannel(endpoint, ch))

def quick_check(wfset:Waveform, idx = 0):
    for idx in range(idx):
        plt.plot(wfset.waveforms[idx].adcs - wfset.waveforms[idx].analyses['std'].result['baseline'])
        plt.ylim(-200, 500)
    plt.legend(title=wfset.waveforms[0].channel)

wfsetch = ChannelWsGrid.clusterize_waveform_set(wfset_full)
    
    

In [ ]:

matplotlib_plot_WaveformSetGrid(wfset_full, quick_check, detector=detectors, func_params=dict(idx=10))
# for a in range(0,10):
#     quick_check(wfsetch[endpoint][16], idx=a)
    